---
title: "Lab 3 - Singing a Song"
author: "Tyler Allen"
date: "2024-10-14"
format:
  html:
    cold-fold: true
    embed-resources: true
theme: "lux"
---

In [1]:
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

In [6]:
xmas.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN


In [7]:
#Function 1: pluralize_gift
def pluralize_gift(gift):
    """
    Returns plural of a noun

    Parameters
    ----------
    gift: str
        A noun

    Return
    ------
    str
        Plural version
    """

    if isinstance(gift, pd.Series):  # Vectorized case for Series (got help from ChatGPT)
        return gift.apply(pluralize_gift)

    # Special cases
    if gift == "partridge":  # Always singular
        return "partridge"
    elif gift == "goose":  # Irregular pluralization
        return "geese"

    #General pluralization rules
    if gift.endswith("y") and gift[-2] not in "aeiou":  # If the word ends with a y
        return gift[:-1] + "ies"
    elif gift.endswith("s") or gift.endswith("x") or gift.endswith("ch") or gift.endswith("sh"):  # Adding 'es'
        return gift + "es"
    else:
        return gift + "s"  # Default

# Tests
test_single_goose = pluralize_gift("goose")  #Should say geese
test_single_partridge = pluralize_gift("partridge")  #Should say partridge

#Testing with entire column from the xmas dataset (vectorized)
pluralized_gifts_column = pluralize_gift(xmas['Gift.Item'])

pluralized_gifts_column.head()

,Gift.Item
0,partridge
1,doves
2,hens
3,birds
4,rings


In [8]:
#Tests
pluralize_gift("goose")

'geese'

In [ ]:
pluralize_gift("dog")

'dogs'

In [ ]:
pluralize_gift("lady")

'ladies'

In [ ]:
pluralize_gift("cat")

'cats'

In [9]:
#Function 2: make_phrase
def make_phrase(num, num_word, item, verb, adjective, location):
    """
    Creates a phrase for the gift given on a particular day of Christmas

    Parameters
    ----------
    num: int
        The day number (1-12)
    num_word: str
        The day number in words (example: "ten")
    item: str
        The gift item
    verb: str or NaN
        The verb associated with the gift (optional)
    adjective: str or NaN
        The adjective describing the gift (optional)
    location: str or NaN
        The location associated with the gift (optional)

    Return
    ------
    str
        A phrase describing the gift for the day
    """

    # Step 1: Replace NAs with blank strings (couldn't figure out how to do it using verb.fillna so I asked ChatGPT for another way)
    verb = "" if pd.isna(verb) else verb
    adjective = "" if pd.isna(adjective) else adjective
    location = "" if pd.isna(location) else location

     # Step 2: If the day number is greater than 1, pluralize the item
    if num > 1:
        item = pluralize_gift(item)

     # Step 3:Check if the item starts with a vowel
    vowels = "aeiou"
    starts_with_vowel = item[0].lower() in vowels

    # Step 4:Handle first day special case (used ChatGPT for help with this)
    if num == 1:
        if starts_with_vowel:
            phrase = f"an {adjective} {item} {location}".strip()
        else:
            phrase = f"a {adjective} {item} {location}".strip()
    else:
        phrase = f"{num_word} {adjective} {item} {verb} {location}".strip()

# Step 5: Return the final phrase
    return " ".join(phrase.split())  # This ensures no extra spaces

#Test
test_phrase = make_phrase(num=10, num_word="ten", item="lord", verb="a leaping", adjective="", location="")
test_phrase

'ten lords a leaping'

In [10]:
#Mapping dictionary to convert day numbers to the desired word form (cardinal numbers)
day_to_word_map = {
    1: "one", 2: "two", 3: "three", 4: "four", 5: "five", 6: "six",
    7: "seven", 8: "eight", 9: "nine", 10: "ten", 11: "eleven", 12: "twelve"
}

# Applying the map to create a new column
xmas['Day.Cardinal'] = xmas['Day'].map(day_to_word_map)

#unit test
xmas.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Day.Cardinal
0,1,first,partridge,NaN,NaN,in a pear tree,one
1,2,second,dove,NaN,turtle,NaN,two
2,3,third,hen,NaN,french,NaN,three
3,4,fourth,bird,NaN,calling,NaN,four
4,5,fifth,ring,NaN,golden,NaN,five


In [25]:
#Testing make_phrase function and applying it to original dataset
xmas['Full.Phrase'] = xmas.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.Cardinal'],
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ), axis=1
)

#Show new column
xmas[['Day', 'Full.Phrase']].head()

,Day,Full.Phrase
0,1,a partridge in a pear tree
1,2,two turtle doves
2,3,three french hens
3,4,four calling birds
4,5,five golden rings


In [31]:
#Creating a mapping for day numbers to their ordinal words (e.g., "1" -> "first")
day_to_ordinal_map = {
    1: "first", 2: "second", 3: "third", 4: "fourth", 5: "fifth", 6: "sixth",
    7: "seventh", 8: "eighth", 9: "ninth", 10: "tenth", 11: "eleventh", 12: "twelfth"
}

In [32]:
#function 3: sing_day
def sing_day(dataset, num, phrase_col):
    """
    Sing the cumulative verses for each day of Christmas up to the given day,
    with commas after every line except the last, which ends with a period.

    Parameters
    ----------
    dataset: DataFrame
        The dataset containing gift phrases.
    num: int
        The day number to sing up to.
    phrase_col: str
        The name of the column containing the phrases.

    Returns
    -------
    str
        The full lyrics for all the days leading up to the given day, each in its own paragraph.
    """

    #created a new function to help with all the formatting and grammar - couldn't figure out how to do it otherwise
    def sing_day_single(dataset, num, phrase_col):
        """
        Sing the phrase for the given day of Christmas with commas and periods.
        """
        # Step 1: Setup the intro line
        num_word = day_to_ordinal_map[num]  # Convert "1" to "first" etc.
        intro = f"On the {num_word} day of Christmas, my true love sent to me:"

        # Step 2: Sing the gift phrases in reverse order from the current day down to 1
        def get_phrase(day):
            phrase = dataset.loc[dataset['Day'] == day, phrase_col].values[0]
            if day == 1 and num > 1:  # Add "and" before the first gift if it's not the first day
                return "and " + phrase
            return phrase

        # Use map for gift phrases for the given day
        gifts = list(map(get_phrase, range(num, 0, -1)))

        # Step 3: Handle the first day differently (no commas)
        if num == 1:
            full_song = f"{intro}\n" + gifts[0] + "."  # First day has no comma
        else:
            # Add commas to all lines except the last one
            full_song = f"{intro}\n"
            full_song += ",\n".join(gifts[:-1]) + ",\n"  # Add commas to all lines except the last one
            full_song += gifts[-1] + "."  # Add a period to the last line

        return full_song

    #Lyrics for each day from 1 to the given day
    full_song = "\n\n".join(sing_day_single(dataset, day, phrase_col) for day in range(1, num + 1))

    return full_song

In [33]:
result = sing_day(xmas, 3, "Full.Phrase")
print(result)

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.


In [35]:
#new dataset
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
xmas2.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN


In [36]:
# Test with the entire column from the xmas2 dataset (vectorized)
pluralized_gifts_column = pluralize_gift(xmas2['Gift.Item'])  # Vectorized operation

pluralized_gifts_column.head()

,Gift.Item
0,emails
1,points
2,pens
3,reviews
4,exams


In [37]:
# Applying the map to create a new column
xmas2['Day.Cardinal'] = xmas2['Day'].map(day_to_word_map)

#unit test
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Day.Cardinal
0,1,first,email,NaN,NaN,from Cal Poly,one
1,2,second,point,NaN,meal,NaN,two
2,3,third,pen,NaN,lost,NaN,three
3,4,fourth,review,NaN,course,NaN,four
4,5,fifth,exam,NaN,practice,NaN,five
5,6,sixth,grader,grading,NaN,NaN,six
6,7,seventh,senior,stressing,NaN,NaN,seven
7,8,eighth,mom,a-calling,NaN,NaN,eight
8,9,ninth,party,bumping,NaN,NaN,nine
9,10,tenth,load,of laundry,NaN,NaN,ten


In [38]:
#Testing make_phrase function and applying it to original dataset
xmas2['Full.Phrase'] = xmas2.apply(
    lambda row: make_phrase(
        num=row['Day'],
        num_word=row['Day.Cardinal'],
        item=row['Gift.Item'],
        verb=row['Verb'],
        adjective=row['Adjective'],
        location=row['Location']
    ), axis=1
)

#Show new column
xmas2[['Day', 'Full.Phrase']].head()

,Day,Full.Phrase
0,1,an email from Cal Poly
1,2,two meal points
2,3,three lost pens
3,4,four course reviews
4,5,five practice exams


In [39]:
#using old sing_day function
result2 = sing_day(xmas2, 12, 'Full.Phrase')
print(result2)

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On t